In [1]:
import json
import functions as f

## Unconditional prompt

In [2]:
# gpt3.5 mbti and api cost(per round) 
result_gpt3 = f.get_mbti(model='gpt-3.5-turbo')
print(json.dumps(result_gpt3))

{"model": "gpt-3.5-turbo", "details": {"E": 38, "I": 25, "S": 19, "N": 62, "T": 35, "F": 34, "J": 51, "P": 15}, "res": "ENTJ", "prompt_tokens": 12678, "input_tokens": 294, "cost": 0.006486000000000004}


In [7]:
middle_tem_gpt3 = f.get_mbti(
    model='gpt-3.5-turbo',
    temperature=1
)

In [8]:
high_tem_gpt3 = f.get_mbti(
    model='gpt-3.5-turbo',
    temperature=1.7
)

In [9]:
unconditional_mbti_gpt3 = {}
unconditional_mbti_gpt3["0.5 temperature unconditional"] = result_gpt3
unconditional_mbti_gpt3["1 temperature unconditional"] = middle_tem_gpt3
unconditional_mbti_gpt3["1.7 temperature unconditional"] = high_tem_gpt3

In [4]:
# gpt4o mbti and api cost(per round)
result_gpt4o = f.get_mbti(model='gpt-4o')
print(json.dumps(result_gpt4o))

{"model": "gpt-4o", "details": {"E": 22, "I": 41, "S": 21, "N": 60, "T": 35, "F": 34, "J": 60, "P": 6}, "res": "INTJ", "prompt_tokens": 12351, "input_tokens": 279, "cost": 0.06314999999999993}


In [11]:
middle_tem_gpt4o = f.get_mbti(
    model='gpt-4o',
    temperature=1
)

In [12]:
high_tem_gpt4o = f.get_mbti(
    model='gpt-4o',
    temperature=1.7
)

In [13]:
unconditional_mbti_gpt4o = {}
unconditional_mbti_gpt4o["0.5 temperature unconditional"] = result_gpt4o
unconditional_mbti_gpt4o["1 temperature unconditional"] = middle_tem_gpt4o
unconditional_mbti_gpt4o["1.7 temperature unconditional"] = high_tem_gpt4o

In [14]:
# Save result of two models using a dict
SAVE_PATH = 'mbti_results/llms_mbti93.json'

llms_mbti = {}
llms_mbti["gpt-3.5"] = unconditional_mbti_gpt3
llms_mbti["gpt-4o"] = unconditional_mbti_gpt4o

# Save the dictionary as a JSON object to the file
with open(SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(llms_mbti, json_file, indent=4)  # indent=4 for pretty-printing

## Conditional prompt (change LLMs' mbti via prompt)

In [61]:
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
# gpt 3.5
personalities = json.load(
    open('mbti_types/personality_traits.json', 'r', encoding='utf8')
).keys()

simple_35_condition_mbti = {}

for p in personalities:

    condi_prompt = (
        'Your personality is'
        + p
    )
    
    result = f.get_mbti(
        model='gpt-3.5-turbo', 
        add_sys_prompt=condi_prompt
    )
    
    simple_35_condition_mbti[p] = result


In [ ]:
# Complex Personality-Conditioned Prompting, 1 loop for 16 personalities
# gpt 3.5
personality_traits = json.load(
    open('mbti_types/personality_traits.json', 'r', encoding='utf8')
)

complex_35_condition_mbti = {}

for p in personality_traits.keys():

    condi_prompt = (
        f'You are a human with the following personality type:{p}.'
        + f'Your traits are the following:{personality_traits[p]["traits"]}'
    )
    
    result = f.get_mbti(
        model='gpt-3.5-turbo', 
        add_sys_prompt=condi_prompt
    )
    
    complex_35_condition_mbti[p] = result

In [ ]:
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
# gpt 4o
personalities = json.load(
    open('mbti_types/personality_traits.json', 'r', encoding='utf8')
).keys()

simple_4o_condition_mbti = {}

for p in personalities:
    
    condi_prompt = (
        'Your personality is'
        + p
    )
    
    result = f.get_mbti(
        model='gpt-4o', 
        add_sys_prompt=condi_prompt
    )
    
    simple_4o_condition_mbti[p] = result


In [ ]:
# Complex Personality-Conditioned Prompting, 1 loop for 16 personalities
# gpt 4o
personality_traits = json.load(
    open('mbti_types/personality_traits.json', 'r', encoding='utf8')
)

complex_4o_condition_mbti = {}

for p in personality_traits.keys():

    condi_prompt = (
        f'You are a human with the following personality type:{p}.'
        + f'Your traits are the following:{personality_traits[p]["traits"]}'
    )
    
    result = f.get_mbti(
        model='gpt-4o', 
        add_sys_prompt=condi_prompt
    )
    
    complex_4o_condition_mbti[p] = result

In [76]:
# save to file
CONDI_SAVE_PATH = 'mbti_results/condition_mbti93.json'

condition_mbti = {}
condition_mbti['simple_35_condition'] = simple_35_condition_mbti
condition_mbti['complex_35_condition'] = complex_35_condition_mbti
condition_mbti['simple_4o_condition'] = simple_4o_condition_mbti
condition_mbti['simple_4o_condition'] = complex_4o_condition_mbti

with open(CONDI_SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(condition_mbti, json_file, indent=4)

In [7]:
# calculate the accuracy
condition_results = json.load(
    open('mbti_results/condition_mbti93.json', 'r', encoding='utf8')
)

accuracy_rate = {}

for r in condition_results.keys(): # r: one of the four conditional mbti results
    temp_acc_num = 0
    temp_dim = [0, 0, 0, 0]
    dim = ['I/E', 'N/S', 'F/T', 'P/J']
    
    for mbti_type in condition_results[r].keys(): # mbti_type: one of 16 MBTI personalities
        res_mbti = condition_results[r][mbti_type]['res'] 
        
        if mbti_type == res_mbti: # total accuracy
            temp_acc_num += 1 
        
        for n in range(4):
            if mbti_type[n] == res_mbti[n]: # accuracy of certain dimesion
                temp_dim[n] += 1  

    accuracy_rate[r] = {} # aggregate dict
    accuracy_rate[r]['total'] = temp_acc_num / 16
    
    for d in range(4):
        accuracy_rate[r][dim[d]] = temp_dim[d] / 16
        
    # print(r + ' done!')


In [8]:
accuracy_rate

{'simple_35_condition': {'total': 0.75,
  'I/E': 1.0,
  'N/S': 0.8125,
  'F/T': 1.0,
  'P/J': 0.9375},
 'complex_35_condition': {'total': 0.875,
  'I/E': 1.0,
  'N/S': 0.9375,
  'F/T': 1.0,
  'P/J': 0.9375},
 'simple_4o_condition': {'total': 0.9375,
  'I/E': 1.0,
  'N/S': 0.9375,
  'F/T': 1.0,
  'P/J': 1.0},
 'complex_4o_condition': {'total': 0.9375,
  'I/E': 1.0,
  'N/S': 0.9375,
  'F/T': 1.0,
  'P/J': 1.0}}